In [17]:
using SummationByParts, LinearAlgebra
using SummationByParts.Cubature, SummationByParts.SymCubatures
using Latexify

# Construct SBP Operators

Okay, we have derived some existing or new quadrature rules. How do we proceed to construct SBP operators?

## SBP Operators on Triangles (with LG or LGL facet nodes)

Consider the quadrature rule we derived for SBP diagonal-E operators.

In [18]:
qf = 11; # the facet quadrature must be qf=q+mod(q,2)+1
cub,_ = SummationByParts.Cubature.quadrature(qf, internal=false); # get LGL nodes for the facet nodes
xedge = cub.params;

In [19]:
cub_tri, vtx_tri = SummationByParts.deriveTriCubatureDiagE(q=9,
                                                    vertices=true, 
                                                    midedges=true, 
                                                    numS21=3, 
                                                    numedge=2, 
                                                    numS111=1, 
                                                    centroid=false,
                                                    delta1=1e-2, delta2=1e-2, xinit = [], xedge = xedge, verbose=false)

----------------------------------------------------------------------------------------------
iter_pso = 800:  iter_lma = 25:  nperturb_pso = 1:  res norm = 2.1698610138775944e-15
----------------------------------------------------------------------------------------------


(TriSymCub{Float64}(7, 8, 33, true, true, false, 2, 3, 1, [0, 5, 3], [0.20874149756969354, 0.5052777031940232, 0.9019135602326743, 0.9151119481392835, 0.7344243967353571, 0.12720911519282319, 0.5890156389073408], [0.001572607990079414, 0.021822749896475195, 0.10552616307685722, 0.19316667114967143, 0.09356963927041088, 0.01791574409676497, 0.014623297206464315, 0.09296537633835696]), [-1.0 -1.0; 1.0 -1.0; -1.0 1.0])

To build SBP diagonal-E opertor using the above quadrature, we proceed as follows:

In [20]:
w, Q, E = SummationByParts.buildoperators(cub_tri, vtx_tri, 5, vertices=true)

([0.001572607990079414, 0.001572607990079414, 0.001572607990079414, 0.021822749896475195, 0.021822749896475195, 0.021822749896475195, 0.10552616307685722, 0.10552616307685722, 0.10552616307685722, 0.19316667114967143  …  0.014623297206464315, 0.014623297206464315, 0.014623297206464315, 0.014623297206464315, 0.09296537633835696, 0.09296537633835696, 0.09296537633835696, 0.09296537633835696, 0.09296537633835696, 0.09296537633835696], [-0.023809523809523794 -0.002388552750029964 … 0.002089742942075226 0.0220523348279083; 0.002388552750029964 0.02380952380952383 … 0.020878627947158224 0.025347107398142443; … ; -0.002089742942075226 -0.020878627947158224 … 0.0 0.030781509272223284; -0.0220523348279083 -0.025347107398142443 … -0.030781509272223284 0.0;;; -0.02380952380952383 -0.0011942763750150192 … 0.0026468315896957073 0.021721125283689635; 0.001194276375014923 3.469446951953614e-17 … -0.00446847945098417 0.0044684794509838735; … ; -0.0026468315896957073 0.00446847945098417 … 0.0 0.0615630

Now we can access all operators as follows:

In [21]:
Qx = Q[:,:,1];
Qy = Q[:,:,2];
H = diagm(w);
Ex = E[:,:,1];
Ey = E[:,:,2];
Dx = inv(H)*Qx;
Dy = inv(H)*Qy;
Sx = Qx - 0.5.*Ex;
Sy = Qy - 0.5.*Ey;

We can check if the SBP property is satisfied.

In [22]:
norm(Qx+Qx' - Ex)

1.9305769020362074e-17

In [23]:
norm(Qy+Qy' - Ey)

1.9305769020362074e-17

## SBP Operators on Tetrahedra

Consider the quadrature rule of degree 6 that we have derived previously to construct a degree $p=3$ SBP diagonal-E operator. 

In [24]:
q=6;
p=3;
qf=2*p;

In [25]:
cub_facet, vtx_facet = SummationByParts.deriveTriCubatureGamma(q=qf,
                                                       vertices=true,
                                                       midedges=false,
                                                       numS21=2,
                                                       numedge=1,
                                                       numS111=0,
                                                       centroid=false,
                                                       xinit=[],
                                                       delta1=1e-3,delta2=1e-1,verbose=false)

No solution found after a 200 LMA iterations.
No solution found after a 200 LMA iterations.
No solution found after a 200 LMA iterations.
No solution found after a 200 LMA iterations.
----------------------------------------------------------------------------------------------
iter_pso = 7200:  iter_lma = 812:  nperturb_pso = 98:  res norm = 1.3771293916064013e-15
----------------------------------------------------------------------------------------------


(TriSymCub{Float64}(3, 4, 15, true, false, false, 1, 2, 0, [0, 3, 1], [0.2372273727931858, 0.8506802519794943, 0.6922540583740083], [0.014260718614408955, 0.20376930605390364, 0.3303589772911329, 0.05913883235361056]), [-1.0 -1.0; 1.0 -1.0; -1.0 1.0])

In [26]:
xedge = cub_facet.params
cub_tet, vtx_tet = SummationByParts.deriveTetCubatureDiagE(q=q,
                                                   vertices=true,
                                                   numS31=1,
                                                   midedges=false, 
                                                   numS22=1,
                                                   numfaceS21=2, 
                                                   numedge=1, 
                                                   numS211=0,
                                                   numfaceS111=0, 
                                                   facecentroid=false,
                                                   numS1111=0,
                                                   centroid=true,
                                                   xinit=[],
                                                   xedge=xedge,
                                                   delta1=1e-3,
                                                   delta2=1e-1,
                                                   verbose=false)

----------------------------------------------------------------------------------------------
iter_pso = 1600:  iter_lma = 12:  nperturb_pso = 5:  res norm = 3.1207043064146366e-14
----------------------------------------------------------------------------------------------


(TetSymCub{Float64}(5, 7, 51, true, false, true, false, 1, 2, 0, 1, 1, 0, 0, [1, 2, 1, 3, 0], [0.39314846687717114, 0.2578639185632058, 0.2372273727931858, 0.8506802519794943, 0.6922540583740083], [0.0008311992138366359, 0.067074516283964, 0.08700624630787392, 0.014487003956738027, 0.02528103240642213, 0.004650827385021338, 0.006646628516709846]), [-1.0 -1.0 -1.0; 1.0 -1.0 -1.0; -1.0 1.0 -1.0; -1.0 -1.0 1.0])

Now, we can simply construct the an SBP operator of degree $p=3$ as follows:

In [27]:
# We must provide the facet operator type, currently the availabe types are, :Omega and :DiagE
# To enforce using the facet quadrature we derived, we must also provide facetcub and facetvtx; 
# otherwise, available default facet operators saved in the code will be used
w, Q, E = SummationByParts.buildoperators(cub_tet, vtx_tet, p, faceopertype=:DiagE, facecub=cub_facet, facevtx=vtx_facet)

([0.0008311992138366359, 0.0008311992138366359, 0.0008311992138366359, 0.0008311992138366359, 0.067074516283964, 0.067074516283964, 0.067074516283964, 0.067074516283964, 0.08700624630787392, 0.08700624630787392  …  0.004650827385021338, 0.004650827385021338, 0.004650827385021338, 0.004650827385021338, 0.004650827385021338, 0.004650827385021338, 0.004650827385021338, 0.004650827385021338, 0.004650827385021338, 0.006646628516709846], [-0.007130359307204477 -0.001196399959226769 … -5.2988919636878614e-5 0.0032641516168204375; 0.001196399959226769 0.007130359307204477 … 5.2988919636859355e-5 -0.0032641516168204804; … ; 5.2988919636878614e-5 -5.2988919636859355e-5 … 0.0 5.262483483100303e-17; -0.0032641516168204375 0.0032641516168204804 … -5.262483483100303e-17 0.0;;; -0.007130359307204477 -0.0005981999796133772 … -0.0027049822264964933 0.003264151616820439; 0.0005981999796133772 0.0 … -0.0026519933068596528 -6.323958535667037e-17; … ; 0.0027049822264964933 0.0026519933068596528 … 0.0295694

Then, we access all the remaining operators as:

In [28]:
Qx = Q[:,:,1];
Qy = Q[:,:,2];
Qz = Q[:,:,3];
H = diagm(w);
Ex = E[:,:,1];
Ey = E[:,:,2];
Ez = E[:,:,3];
Dx = inv(H)*Qx;
Dy = inv(H)*Qy;
Dz = inv(H)*Qz;
Sx = Qx - 0.5.*Ex;
Sy = Qy - 0.5.*Ey;
Sz = Qz - 0.5.*Ez;

Check if the SBP property is satisfied:

In [29]:
norm(Qx+Qx' - Ex)

0.0

In [30]:
norm(Qy+Qy' - Ey)

0.0

In [31]:
norm(Qz+Qz' - Ez)

0.0